[RAG-Evaluation-Dataset-JA](https://huggingface.co/datasets/allganize/RAG-Evaluation-Dataset-JA)に沿ってコンテキストとなる日本語データを集め、評価用データもダウンロードしました。


In [5]:
!wget -P ../data https://huggingface.co/datasets/allganize/RAG-Evaluation-Dataset-JA/resolve/main/documents.csv 

--2025-02-07 10:16:22--  https://huggingface.co/datasets/allganize/RAG-Evaluation-Dataset-JA/resolve/main/documents.csv
huggingface.co (huggingface.co) をDNSに問いあわせています... 2600:9000:27af:8800:17:b174:6d00:93a1, 2600:9000:27af:d200:17:b174:6d00:93a1, 2600:9000:27af:2e00:17:b174:6d00:93a1, ...
huggingface.co (huggingface.co)|2600:9000:27af:8800:17:b174:6d00:93a1|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 13161 (13K) [text/plain]
`../data/documents.csv.1' に保存中

documents.csv.1     100%[===================>]  12.85K  --.-KB/s 時間 0s         

2025-02-07 10:16:22 (1.53 GB/s) - `../data/documents.csv.1' へ保存完了 [13161/13161]



In [1]:
import pandas as pd
df = pd.read_csv("../data/documents.csv", encoding="utf-8")
df.head()

,domain,title,page,url,file_name,publisher
0,finance,2021（令和3）年度 生命保険に関する全国実態調査,17,https://www.jili.or.jp/files/research/zenkokuj...,i-xvii.pdf,公益財団法人生命保険文化センター
1,finance,2023年 保険モニタリングレポート【概要】,11,https://www.fsa.go.jp/news/r4/hoken/20230630-2...,01.pdf,金融庁
2,finance,我が国における気候関連リスクによる住宅ローン・ポートフォリオへの影響分析,31,https://www.fsa.go.jp/frtc/seika/discussion/20...,DP2023-2.pdf,金融庁
3,finance,2023事務年度 金融行政方針,30,https://www.fsa.go.jp/news/r5/20230829/230829_...,230829_main.pdf,金融庁
4,finance,２０２３年度株式分布状況調査の調査結果について,19,https://www.jpx.co.jp/markets/statistics-equit...,j-bunpu2023.pdf,株式会社日本取引所グループ


In [2]:
import re
from pathlib import Path
import requests
import pandas as pd

def sanitize_filename(filename: str) -> str:
    # ファイル名に使用できない文字を除去
    return re.sub(r'[\\/*?:"<>|]', '_', filename)

# CSVファイルの読み込み
df = pd.read_csv("../data/documents.csv", encoding="utf-8")

base_dir = Path("../data/pdf")
base_dir.mkdir(parents=True, exist_ok=True)

# ブラウザの User-Agent を設定
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}

# timeoutの設定
timeout_duration = 10

# itertuples を利用して各行を処理（iには1からの連番を付与）
for i, row in enumerate(df.itertuples(index=False), start=0): 
    # 各カラムの値を取得（rowは属性でアクセス可能）
    domain = row.domain
    url = row.url
    title = row.title
    file_name = row.file_name
    # print(i, url)
    # タイトルをサニタイズして、番号付きのファイル名を生成
    if domain == "public":
        sanitized_title = sanitize_filename(title)
        file_path = base_dir / f"{i}_{domain}_{sanitized_title}_file_name={file_name}"

        try:
            response = requests.get(url, timeout=timeout_duration, headers=headers)
            response.raise_for_status()
        except Exception as e:
            print(f"ダウンロード失敗: URL = {url}, エラー: {e}")
            continue
            
        # Content-Type が PDF であることのチェックを追加
        content_type = response.headers.get("Content-Type", "").lower()
        if not content_type.startswith("application/pdf"):
            print(f"PDF ではない可能性: URL = {url} 内容タイプ: {content_type}")
            continue

        file_path.write_bytes(response.content)
        print(f"保存完了: {file_path}")

保存完了: ../data/pdf/53_public_神奈川県企業庁における総合評価方式に関する運用ガイドライン〔建設工事編〕（令和6年6月）_file_name=r06_06_guideline_itaku.pdf
保存完了: ../data/pdf/54_public_自治体の行政手続のオンライン化に係る手順書【第2.0版】_file_name=000857916.pdf
保存完了: ../data/pdf/55_public_デフレ完全脱却のための総合経済対策～日本経済の新たなステージにむけて～（令和５年11月2日）施策例（全体版3）_file_name=20231102_sesaku3.pdf
保存完了: ../data/pdf/56_public_保健所の活用の仕方_file_name=000761091.pdf
保存完了: ../data/pdf/57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf
保存完了: ../data/pdf/58_public_地方公共団体の基幹業務システムの統一・標準化 標準仕様書と適合確認に関する考え方_file_name=20231027_policies_local_governments_outline_02.pdf
保存完了: ../data/pdf/59_public_ふるさと納税に係る指定制度の運用について_file_name=000956094.pdf
保存完了: ../data/pdf/60_public_インボイス制度の負担軽減措置のよくある質問とその回答（令和５年３⽉ 31 ⽇時点）_file_name=qa_futankeigen.pdf
保存完了: ../data/pdf/61_public_「臨海副都心まちづくりガイドライン－2022改定－」（令和4年1月）_file_name=rinpukuGL_2022.pdf
保存完了: ../data/pdf/62_public_地域脱炭素ロードマップ～地方からはじまる、次の時代への移行戦略～_file_name=①地域脱炭素ロードマップ.pdf
保存完了: ../data/pdf/63_public_2023（令和５）年 国民生活基礎調査の概況_file_name=10.pd

In [4]:
%pip install -qU datasets 
from datasets import load_dataset

ds = load_dataset("allganize/RAG-Evaluation-Dataset-JA")

Note: you may need to restart the kernel to use updated packages.


/Users/kj/.pyenv/versions/3.10.10/envs/env_py3_10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
eval_data = ds['test']
tgt_name = "data/pdf/64_public_訪日外国人の消費動向（2024 年 1-3 月期 報告書）_file_name=001750938.pdf".split("file_name=")[-1]
print(tgt_name)
tgt = eval_data.filter(lambda x: x["target_file_name"]==tgt_name)
tgt[-1]

001750938.pdf


{'question': '訪日外国人が目的を問わず買い物をした場所について、空港の免税店を除き最も利用された場所の上位5つと、利用率の数値を教えてください。',
 'target_answer': '多い順に、コンビニエンスストア（83.6%）、ドラッグストア（56.0%）、百貨店・デパート（55.6%）、スーパーマーケット（47.6%）、観光地の土産店（36.5%）となっています。',
 'target_file_name': '001750938.pdf',
 'target_page_no': 23,
 'domain': 'public',
 'type': 'image',
 'alli_gpt-4o_answer': '訪日外国人が目的を問わず買い物をした場所について、空港の免税店を除いた最も利用された場所の上位5つとその利用率は以下の通りです。    1. コンビニエンスストア - 83.6%  2. ドラッグストア - 56.0%  3. 百貨店・デパート - 55.6%  4. スーパーマーケット - 47.6%  5. 観光地の土産店 - 36.5%    (参考: context 2)',
 'alli_gpt-4o_ox': 'O',
 'alli_gpt-4_answer': '訪日外国人が最も利用した買い物場所の上位5つは、「コンビニエンスストア」(83.6%)、「空港の免税店」(60.7%)、「ドラッグストア」(56.0%)、「百貨店・デパート」(55.6%)、「スーパーマーケット」(47.6%)です。ただし、空港の免税店を除くと、「コンビニエンスストア」(83.6%)、「ドラッグストア」(56.0%)、「百貨店・デパート」(55.6%)、「スーパーマーケット」(47.6%)、「ディスカウントストア」(25.6%)となります[2]。',
 'alli_gpt-4_ox': 'X',
 'alli_gpt-4-turbo_answer': '訪日外国人が目的を問わず買い物をした場所で、空港の免税店を除いた上位5つの場所とその利用率は以下の通りです：    1. コンビニエンスストア: 83.6%  2. ドラッグストア: 56.0%  3. 百貨店・デパート: 55.6%  4. スーパーマーケット: 47.6%  5. 観光地の土産店

In [31]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("../data/pdf/64_public_訪日外国人の消費動向（2024 年 1-3 月期 報告書）_001750938.pdf")
docs = loader.load_and_split()
docs

[Document(metadata={'source': '../data/pdf/64_public_訪日外国人の消費動向（2024 年 1-3 月期 報告書）_001750938.pdf', 'page': 0, 'page_label': '1'}, page_content='訪日外国人の消費動向 \n訪日外国人消費動向調査結果及び分析 \n \n2024 年 1-3 月期  報告書 \n \n \n \n \n \n \n \n \n \n \n \n \nJAPAN TOURISM AGENCY'),
 Document(metadata={'source': '../data/pdf/64_public_訪日外国人の消費動向（2024 年 1-3 月期 報告書）_001750938.pdf', 'page': 1, 'page_label': '2'}, page_content='訪日外国人消費動向調査'),
 Document(metadata={'source': '../data/pdf/64_public_訪日外国人の消費動向（2024 年 1-3 月期 報告書）_001750938.pdf', 'page': 2, 'page_label': '3'}, page_content='2024 年 1-3 月期 報告書  \n \n訪日外国人の消費動向 \n2024年1-3月期 報告書 \n \n目 次 \n \n \nⅠ  訪日外国人の消費動向の分析結果 \n１．訪日外国人の属性と 旅行内容 \n（１）全目的 ･･････････････････････････  3 \n（２）観光・レジャー目的 ･････････････ 9 \n \n２．訪日外国人の旅行支出  \n（１）旅行支出の構造 ･････････････････ 13 \n（２）旅行前支出 ･････････････････････ 14 \n（３）旅行中支出 ･････････････････････ 14 \n（４）旅行支出の推計 ･････････････････ 16 \n（５）費目別にみる旅行支出 ･･･････････ 18 \n（６）来訪目的別にみる旅行支出 ･･･････ 19 \n（７）クルーズ客の旅行中支出･･････････ 19 \n \n３．土産品